In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
base_dir = os.getenv('BASEDIR')
from pathlib import Path
import re
import pickle as pk
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score
# from lightgbm import LGBMClassifier
from flaml.default import LGBMClassifier

In [2]:
from tqdm import tqdm
tqdm.pandas()

In [3]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [4]:
# datasets = ['qanda', 'ausvotes','socialsense','riot', 'parler']
# datasets = ['qanda']
dataset='qanda'

In [5]:
datasets_twitter = {'qanda', 'ausvotes', 'riot'}

In [6]:
gt_fr_name = 'URLb_FR'
features_lr=['use']
features_fr=['use']

granularity='_per_user'
gt='URL_LR'
data_path = os.path.join(base_dir,'data','01_raw_data',dataset, dataset+granularity+'.pk')
with open(data_path, 'rb') as rf:
    data = pk.load(rf).reset_index(drop=False)

def load_vector(fp):
    with open(fp, 'rb') as rf:
        vec = pk.load(rf)
    return vec

feature_paths_lr = [os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk') for feature in features_lr]
features_lr = np.asarray(np.hstack([load_vector(fp) for fp in feature_paths_lr]))

feature_paths_fr = [os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk') for feature in features_fr]
features_fr = np.asarray(np.hstack([load_vector(fp) for fp in feature_paths_fr]))

gt_stance = load_vector(os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_URL_LR_'+granularity+'.pk'))
gt_fr = load_vector(os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+gt_fr_name+'_'+granularity+'.pk'))    

# Left-Right
# et = LGBMClassifier(n_estimators=100, min_data_in_leaf=500,colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
et_lr = LGBMClassifier(is_unbalance=True,n_estimators=200, n_jobs=-1, verbose=-1, seed=123)
mask_stance = gt_stance >=0
y_stance = gt_stance[mask_stance]
X_stance = features_lr[mask_stance]
et_lr.fit(X_stance,y_stance)
predicted_stance = et_lr.predict(features_lr)

# Far-Right
# et = LGBMClassifier(n_estimators=100, min_data_in_leaf=500,colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
et_fr = LGBMClassifier(is_unbalance=True, n_jobs=-1, verbose=-1, seed=123)
# et = LGBMClassifier(n_estimators=200, n_jobs=-1, verbose=-1, seed=123)
mask_fr = gt_fr >=0
y_fr = gt_fr[mask_fr]
X_fr = features_fr[mask_fr]
et_fr.fit(X_fr,y_fr)
predicted_fr = et_fr.predict(features_fr)

pred_stance = pd.Series([['N', 'L', 'R'][i] for i in predicted_stance]).values

label = pred_stance
label[predicted_fr] = 'FR'
data['label'] = label

In [7]:
with open(os.path.join(base_dir,'data','01_raw_data',dataset,dataset+'_per_post.pk'), 'rb') as rf:
    data_per_post = pk.load(rf)

In [8]:
user_labels = data[['uid', 'label']]

In [9]:
data_per_post['user_propagated_label'] = pd.merge(data_per_post[['uid']], user_labels, how='left', on='uid')['label']

In [10]:
import pandas as pd
import os 
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import tensorflow as tf
import tensorflow_hub as hub

2023-04-28 12:30:08.868203: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 12:30:09.009675: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-28 12:30:09.016642: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roram/anaconda3/lib:
2023-04-28 12:30:09.016656: I tensorflow/stream_executor/cuda/cudart_

In [11]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
    return model(input)

def batched_embed(l_text):
    all_embeddings = []
    chunk_size = 10
    for i in tqdm(range(0, len(l_text), chunk_size)):
        chunk = l_text[i: min(i+chunk_size, len(l_text))]
        emb = embed(chunk)
        all_embeddings.append(emb.numpy())
    return np.vstack(all_embeddings)

2023-04-28 12:32:25.597534: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roram/anaconda3/lib:
2023-04-28 12:32:25.599880: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roram/anaconda3/lib:
2023-04-28 12:32:25.601996: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roram/anaconda3/lib:
2023-04-28 12:32:25.604096: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open s

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [12]:
use_embeddings = batched_embed(list(data_per_post['text']))
features_lr = use_embeddings
features_fr = use_embeddings

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125213/125213 [03:33<00:00, 586.12it/s]


In [13]:
predicted_stance = et_lr.predict(features_lr)
predicted_fr = et_fr.predict(features_fr)

In [14]:
pred_stance = pd.Series([['N', 'L', 'R'][i] for i in predicted_stance]).values

label = pred_stance
label[predicted_fr] = 'FR'
data_per_post['user_based_label'] = label

In [15]:
domain_lr = pd.read_csv(os.path.join(base_dir,'data','02_ground_truth_data','url_data','domain_allsides_reuters_lr.csv')).set_index('domain')

import tldextract
def extract_domain(url):
    ext = tldextract.extract(url)
    return('.'.join([ext.domain, ext.suffix]))

def handle_labels(s):
    if np.isnan(s):
        return -1
    elif s > 0:
        return 2
    elif s == 0:
        return 0
    elif s < -1*0:
        return 1
    else:
        return -1
    
def get_url_ideology(full_url):
    domain = extract_domain(full_url)
    try:
        return(domain_lr.loc[domain].stance)
    except Exception as e:
        return(np.nan)

In [16]:
gt_stance = data_per_post['urls'].progress_apply(lambda l: np.nanmean([get_url_ideology(e) for e in l])).apply(handle_labels)

  0%|                                                                                                                                                                              | 0/1252122 [00:00<?, ?it/s]/tmp/ipykernel_221119/854998142.py:1: RuntimeWarning: Mean of empty slice
  gt_stance = data_per_post['urls'].progress_apply(lambda l: np.nanmean([get_url_ideology(e) for e in l])).apply(handle_labels)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1252122/1252122 [01:03<00:00, 19672.86it/s]


In [17]:
mbfc = pd.read_csv(os.path.join(base_dir,'data','02_ground_truth_data','url_data','mbfc','mbfc_poltical.tsv'), delimiter='\t')
mbfc = pd.read_csv(os.path.join(base_dir,'data','02_ground_truth_data','url_data','mbfc','mbfc_poltical.tsv'), delimiter='\t')
mbfc = mbfc[mbfc['bias'] == 'right']
mbfc['domain'] = mbfc['source_url_normalized']
mbfc['stance'] = 1.0
mbfc = mbfc.set_index('domain')
import tldextract
def extract_domain(url):
    ext = tldextract.extract(url)
    return('.'.join([ext.domain, ext.suffix]))

def handle_labels(s):
    return s > 0.5
    
def get_url_ideology(full_url):
    domain = extract_domain(full_url)
    try:
        return(mbfc.loc[domain].stance)
    except Exception as e:
        return(np.nan)

In [18]:
gt_fr = data_per_post['urls'].progress_apply(lambda l: np.nanmean([get_url_ideology(e) for e in l])).apply(handle_labels)

  0%|                                                                                                                                                                              | 0/1252122 [00:00<?, ?it/s]/tmp/ipykernel_221119/1023213283.py:1: RuntimeWarning: Mean of empty slice
  gt_fr = data_per_post['urls'].progress_apply(lambda l: np.nanmean([get_url_ideology(e) for e in l])).apply(handle_labels)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1252122/1252122 [01:01<00:00, 20242.94it/s]


In [19]:

# Left-Right
# et = LGBMClassifier(n_estimators=100, min_data_in_leaf=500,colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
et_lr = LGBMClassifier(is_unbalance=True,n_estimators=200, n_jobs=-1, verbose=-1, seed=123)
mask_stance = gt_stance >=0
y_stance = gt_stance[mask_stance]
X_stance = features_lr[mask_stance]
et_lr.fit(X_stance,y_stance)
predicted_stance = et_lr.predict(features_lr)

# Far-Right
# et = LGBMClassifier(n_estimators=100, min_data_in_leaf=500,colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
et_fr = LGBMClassifier(is_unbalance=True, n_jobs=-1, verbose=-1, seed=123)
# et = LGBMClassifier(n_estimators=200, n_jobs=-1, verbose=-1, seed=123)
mask_fr = gt_fr >=0
y_fr = gt_fr[mask_fr]
X_fr = features_fr[mask_fr]
et_fr.fit(X_fr,y_fr)
predicted_fr = et_fr.predict(features_fr)

pred_stance = pd.Series([['N', 'L', 'R'][i] for i in predicted_stance]).values

label = pred_stance
label[predicted_fr] = 'FR'
data_per_post['tweet_based_label'] = label

INFO:flaml.default.suggest:metafeature distance: 0.7131673427388265
INFO:flaml.default.suggest:metafeature distance: 18.63728940022159


In [20]:
Path(os.path.join(base_dir,'data','04_results','labels_emma')).mkdir( parents=True, exist_ok=True )
data_per_post[['tid','uid','user_propagated_label', 'user_based_label', 'tweet_based_label']].to_csv(os.path.join(base_dir,'data','04_results','labels_emma',dataset+'.csv'))

In [22]:
data_per_post[['tid','uid','user_propagated_label', 'user_based_label', 'tweet_based_label']].dtypes

tid                      object
uid                      object
user_propagated_label    object
user_based_label         object
tweet_based_label        object
dtype: object

In [ ]:
gt_fr_name = 'URLb_FR'
features_lr=['use']
features_fr=['use']

granularity='_per_user'
gt='URL_LR'
data_path = os.path.join(base_dir,'data','01_raw_data',dataset, dataset+granularity+'.pk')
with open(data_path, 'rb') as rf:
    data = pk.load(rf).reset_index(drop=False)

def load_vector(fp):
    with open(fp, 'rb') as rf:
        vec = pk.load(rf)
    return vec

feature_paths_lr = [os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk') for feature in features_lr]
features_lr = np.asarray(np.hstack([load_vector(fp) for fp in feature_paths_lr]))

feature_paths_fr = [os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk') for feature in features_fr]
features_fr = np.asarray(np.hstack([load_vector(fp) for fp in feature_paths_fr]))

gt_stance = load_vector(os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_URL_LR_'+granularity+'.pk'))
gt_fr = load_vector(os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+gt_fr_name+'_'+granularity+'.pk'))    

# Left-Right
# et = LGBMClassifier(n_estimators=100, min_data_in_leaf=500,colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
et_lr = LGBMClassifier(is_unbalance=True,n_estimators=200, n_jobs=-1, verbose=-1, seed=123)
mask_stance = gt_stance >=0
y_stance = gt_stance[mask_stance]
X_stance = features_lr[mask_stance]
et_lr.fit(X_stance,y_stance)
predicted_stance = et_lr.predict(features_lr)

# Far-Right
# et = LGBMClassifier(n_estimators=100, min_data_in_leaf=500,colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
et_fr = LGBMClassifier(is_unbalance=True, n_jobs=-1, verbose=-1, seed=123)
# et = LGBMClassifier(n_estimators=200, n_jobs=-1, verbose=-1, seed=123)
mask_fr = gt_fr >=0
y_fr = gt_fr[mask_fr]
X_fr = features_fr[mask_fr]
et_fr.fit(X_fr,y_fr)
predicted_fr = et_fr.predict(features_fr)

pred_stance = pd.Series([['N', 'L', 'R'][i] for i in predicted_stance]).values

label = pred_stance
label[predicted_fr] = 'FR'
data['label'] = label

In [ ]:
Path(os.path.join(base_dir,'data','04_results','labels_emma')).mkdir( parents=True, exist_ok=True )
data[['uid','label']].to_csv(os.path.join(base_dir,'data','04_results','labels_emma',dataset+'.csv'))

In [17]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
base_dir = os.getenv('BASEDIR')
from pathlib import Path
import re
import pickle as pk
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score
# from lightgbm import LGBMClassifier
from flaml.default import LGBMClassifier

from tqdm import tqdm

In [18]:
datasets = ['qanda', 'ausvotes','socialsense','riot', 'parler']
# datasets = ['riot']

In [19]:
datasets_twitter = {'qanda', 'ausvotes', 'riot'}

In [20]:
gt_fr_name = 'URLb_FR'
for dataset in tqdm(datasets):
    if dataset in datasets_twitter:
        features_lr=['use', 'rt']
        features_fr=['use', 'ht']
        # features_fr= ['use']
    else:
        features_lr= ['use']
        features_fr= ['use']
    
    granularity='_per_user'
    gt='URL_LR'
    data_path = os.path.join(base_dir,'data','01_raw_data',dataset, dataset+granularity+'.pk')
    with open(data_path, 'rb') as rf:
        data = pk.load(rf).reset_index(drop=False)
        
    def load_vector(fp):
        with open(fp, 'rb') as rf:
            vec = pk.load(rf)
        return vec
    
    feature_paths_lr = [os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk') for feature in features_lr]
    features_lr = np.asarray(np.hstack([load_vector(fp) for fp in feature_paths_lr]))
    
    feature_paths_fr = [os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk') for feature in features_fr]
    features_fr = np.asarray(np.hstack([load_vector(fp) for fp in feature_paths_fr]))
    
    gt_stance = load_vector(os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_URL_LR_'+granularity+'.pk'))
    gt_fr = load_vector(os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+gt_fr_name+'_'+granularity+'.pk'))    

    # Left-Right
    et = LGBMClassifier(is_unbalance=True,n_estimators=200, n_jobs=-1, verbose=-1, seed=41)
    mask_stance = gt_stance >=0
    y_stance = gt_stance[mask_stance]
    X_stance = features_lr[mask_stance]
    print('Training:', 'lr')
    et.fit(X_stance,y_stance)
    print('Trained:', 'lr')
    predicted_stance = et.predict(features_lr)
    predicted_stance_proba = et.predict_proba(features_lr)
    
    # Far-Right
    et = LGBMClassifier(is_unbalance=True, n_jobs=-1, verbose=-1, seed=123)
    mask_fr = gt_fr >=0
    y_fr = gt_fr[mask_fr]
    X_fr = features_fr[mask_fr]
    print('Training:', 'fr')
    et.fit(X_fr,y_fr)
    print('Trained:', 'fr')
    predicted_fr = et.predict(features_fr)
    predicted_fr_proba = et.predict_proba(features_fr)
                        
    pred_stance = pd.Series([['N', 'L', 'R'][i] for i in predicted_stance]).values
    
    label = pred_stance
    label[predicted_fr] = 'FR'
    data['label'] = label
    data['lr_prob_left'] = predicted_stance_proba[:,1]

    data['lr_prob_right'] = predicted_stance_proba[:,2]

    data['lr_label'] = pd.Series([['N', 'L', 'R'][i] for i in predicted_stance]).values

    data['fr_prob'] = predicted_fr_proba[:,1]

    data['fr_label'] = predicted_fr

    Path(os.path.join(base_dir,'data','04_results','labels')).mkdir( parents=True, exist_ok=True )
    data[['uid','label','lr_prob_left','lr_prob_right', 'lr_label', 'fr_prob', 'fr_label']].to_csv(os.path.join(base_dir,'data','04_results','labels',dataset+'_extended.csv'))

  0%|                                                                                                                                                                                    | 0/5 [00:00<?, ?it/s]

Training: lr
Trained: lr
Training: fr
Trained: fr


  0%|                                                                                                                                                                                  | 0/5 [1:09:41<?, ?it/s]


ValueError: Wrong number of items passed 3, placement implies 1

In [40]:
data['lr_prob_left'] = predicted_stance_proba[:,1]

data['lr_prob_right'] = predicted_stance_proba[:,2]

data['lr_label'] = pd.Series([['N', 'L', 'R'][i] for i in predicted_stance]).values

data['fr_prob'] = predicted_fr_proba[:,1]

data['fr_label'] = predicted_fr

Path(os.path.join(base_dir,'data','04_results','labels')).mkdir( parents=True, exist_ok=True )
data[['uid','label','lr_prob_left','lr_prob_right', 'lr_label', 'fr_prob', 'fr_label']].to_csv(os.path.join(base_dir,'data','04_results','labels',dataset+'_extended.csv'))

In [24]:
predicted_stance

array([0, 1, 1, ..., 1, 1, 1])

In [27]:
predicted_stance_proba[:,1]

array([0.29330231, 0.99298013, 0.99981708, ..., 0.99591776, 0.9874386 ,
       0.99372335])

In [37]:
data['lr_prob_left'] = predicted_stance_proba[:,1]

data['lr_prob_right'] = predicted_stance_proba[:,2]

data['lr_label'] = pred_stance

data['fr_prob'] = predicted_fr_proba[:,1]

data['fr_label'] = predicted_fr

Path(os.path.join(base_dir,'data','04_results','labels')).mkdir( parents=True, exist_ok=True )
data[['uid','label','lr_prob_left','lr_prob_right', 'lr_label', 'fr_prob', 'fr_label']].to_csv(os.path.join(base_dir,'data','04_results','labels',dataset+'_extended.csv'))

In [38]:
data

,uid,level_1,text,hashtags,rid,urls,label,lr_prob_left,lr_prob_right,lr_label,fr_prob,fr_label
0,1000001245946490880,0,here we come!! is with 1st March! on no...,"[Wolverhampton, screening, QandA, Tickets, sal...",[1225781455386796033],[http://light-house.co.uk/LightHouseMedi],N,0.293302,0.003465,N,0.000076,False
1,1000022700092411904,0,when will you invite on Australian childre...,"[qanda, GretaThunberg]",[1236486564550565888],[],L,0.992980,0.000464,L,0.000237,False
2,1000111986917572608,0,My daughter was lucky to finish her diploma bu...,"[QandA, qanda, universities, QandA, qanda, shu...","[1297865209139228672, 1300401525051645953, 122...",[],L,0.999817,0.000127,L,0.000214,False
3,1000127207048728576,0,Will you be one of the 99.99% of Australians n...,"[blacklivesmatteraustralia, auspol, qanda, 9to...",[1269083271930896384],[],L,0.793648,0.014002,L,0.000277,False
4,1000175125608583168,0,wheeling out clickbait promos that feed a def...,"[QandA, aussieED, qanda, aussieED, qanda, wome...","[1235133197601492994, 1235107041477156865, 123...",[https://www.abc.net.au/qanda/2020-27-04/12165...,L,0.999689,0.000066,L,0.000810,False
...,...,...,...,...,...,...,...,...,...,...,...,...
103069,999931831364435968,0,RT Meyne Wyatt closes with a monologue from ...,[QandA],[1269982640494411776],[],L,0.997396,0.000515,L,0.000039,False
103070,999937171766235136,0,I said prior to this particular Q&amp;A sh...,[DefundTheABC],"[1270989483844841473, 1270989483844841473, 130...",[],L,0.974676,0.022946,L,0.040587,False
103071,999954234119356416,0,I know some folks are miffed I didn't get more...,[QandA],"[1224309168909082625, 1269982640494411776]",[],L,0.995918,0.000132,L,0.000085,False
103072,999960633960316929,0,RT Rio Tinto “oops we didnt know it was that ...,"[qanda, QandA, auspol2020]","[1267430023620788227, 1320679824331993088]",[],L,0.987439,0.001694,L,0.000108,False
